In [16]:
import xml.etree.ElementTree as ET
import re

def to_tei(xml_entree, chemin_sortie):
        
    entree_tree = ET.parse(xml_entree)
    entree_root = entree_tree.getroot()
    
    facslist = []
    transcriptions = {}
    linetype = {}
    capitalflag = False
    headflag = False
    sectionflag = False
    paragflag = False
    
    # DEBUT DE L'ECRITURE DU XML DE SORTIE
    
    # Création de la racine.
    sortie_root = ET.Element("TEI")
    capitale = ET.Element("c")
    head = ET.Element("head")
    
    # Récupération du modèle de teiHeader depuis un fichier externe.
    fichier_teiHeader = ET.parse('data/teiHeader.xml')
    teiHeader = fichier_teiHeader.getroot()
    sortie_root.append(teiHeader)
    
    for facsimile in entree_root.findall('./facsimile'):
        sortie_root.append(facsimile)
    
    for line in entree_root.findall('.//l'):
        facs = line.get('facs')
        transcriptions[facs] = line.text
    
    # Faire un dictionnaire avec les identifiants.
    for ligne in entree_root.findall('.//zone[@rendition="Line"]'):
        facs = '#' + ligne.get('identifiant')
        facslist.append(facs)
        
        if ligne.get('subtype'):
            linetype[facs] = str(ligne.get('subtype'))
        else:
            linetype[facs] = "texte"
                
    body = ET.Element("body")
    
    section = ET.Element("div", subtype="section")
    parag = ET.Element("p")
    
    for facs in facslist:
                
        if linetype[facs] == "heading":
            ligne = ET.Element("l")
            ligne.set('facs', facs)
            ligne.text = transcriptions[facs]
            
            if headflag == True:
                head.append(ligne)
            else:
                section.append(parag)
                del parag
                parag = ET.Element("p")
                
                body.append(section)
                del section
                section = ET.Element("div", subtype="section")
                    
                headflag = True
                head = ET.Element("head")
                head.append(ligne)
                                
        elif linetype[facs] == "drop-capital":
            
            if headflag == True:
                section.append(head)
                del head
                headflag = False
                parag = ET.Element("p")
                
            capitalflag = True
            del capitale
            capitale = ET.Element("c")
            capitale.set('facs', facs)
            capitale.text = transcriptions[facs]
                        
        elif linetype[facs] == "texte":
            ligne = ET.Element("l")
            ligne.set('facs', facs)
            
            if capitalflag == True:
                capitale.tail = transcriptions[facs]
                ligne.append(capitale)
                capitalflag = False
                parag.append(ligne)
            
            else:
                ligne.text = transcriptions[facs]
                parag.append(ligne)
                
                
    text = ET.Element("text")
    text.append(body)
    sortie_root.append(text)
                
    
    
    a_ecrire = ET.tostring(sortie_root, encoding="unicode", method="xml")
    
    ecriture = open(chemin_sortie, "w")
    ecriture.write(a_ecrire)

In [17]:
to_tei('/home/erminea/Documents/CONDE/TAC_export_TS/TAC_TEI_TS/TAC_TEI_TS_tei.xml',
      '/home/erminea/Documents/CONDE/TAC_export_TS/TAC_TEI_TS/TAC_transf.xml')